# 01 – Data Processing: Universe & Daily Excess Returns

This notebook mirrors the original `data_processing` notebook, but now uses
the `stat_arb` Python package:

- Load Bloomberg yearly universes using `data_pipeline.load_bloomberg_universe_excel` / `build_year2blg`.
- Construct the union of tickers across years.
- Download daily **excess** returns (stock – risk-free) via `data_pipeline.get_daily_returns`.
- Build a wide returns panel and save it to `data/processed/` for later notebooks.

### Load paths config and set directories

In [15]:
from pathlib import Path
import pandas as pd

from stat_arb import config, data_pipeline
from stat_arb import utils

pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", 10)

Matplotlib is building the font cache; this may take a moment.


In [16]:
paths_cfg = config.get_paths_config()

data_cfg = paths_cfg.get("data", {})
raw_dir = Path(data_cfg.get("raw_dir", "data/raw"))
processed_dir = Path(data_cfg.get("processed_dir", "data/processed"))

bloomberg_cfg = data_cfg.get("bloomberg", {})
years = bloomberg_cfg.get("years", [2020, 2021, 2022, 2023, 2024, 2025])
pattern = bloomberg_cfg.get("pattern", "{year}0101_US_Port.xlsx")

panel_name = data_cfg.get("panel_name", "daily_excess_returns.parquet")
panel_path = processed_dir / panel_name

processed_dir.mkdir(parents=True, exist_ok=True)
raw_dir.mkdir(parents=True, exist_ok=True)

raw_dir, processed_dir, panel_path

(PosixPath('data/raw'),
 PosixPath('data/processed'),
 PosixPath('data/processed/daily_excess_returns.parquet'))

### Load  Bloomberg universes


In [30]:
year2blg = {}

for yr in years:
    fn = raw_dir / pattern.format(year=yr)
    if not fn.exists():
        print(f"[WARN] Missing Bloomberg file for {yr}: {fn}")
        continue

    print(f"Loading Bloomberg universe for {yr} from {fn}")
    year2blg[yr] = data_pipeline.load_bloomberg_universe_excel(fn)

len(year2blg)

Loading Bloomberg universe for 2020 from data/raw/20200101_US_Port.xlsx
Loading Bloomberg universe for 2021 from data/raw/20210101_US_Port.xlsx
Loading Bloomberg universe for 2022 from data/raw/20220101_US_Port.xlsx
Loading Bloomberg universe for 2023 from data/raw/20230101_US_Port.xlsx
Loading Bloomberg universe for 2024 from data/raw/20240101_US_Port.xlsx
Loading Bloomberg universe for 2025 from data/raw/20250101_US_Port.xlsx


6

### Quick sanity check on universe


In [31]:
summary_rows = []
for yr, df in year2blg.items():
    summary_rows.append(
        {
            "year": yr,
            "num_names": len(df),
            "min_mkt_cap": df["Market_Cap"].min() if "Market_Cap" in df.columns else None,
            "max_mkt_cap": df["Market_Cap"].max() if "Market_Cap" in df.columns else None,
        }
    )

pd.DataFrame(summary_rows).set_index("year")

,num_names,min_mkt_cap,max_mkt_cap
year,,,
2020,900,3.369691e+09,1.304765e+12
2021,903,3.537506e+09,2.255969e+12
2022,865,2.654036e+09,2.913284e+12
2023,871,2.614116e+09,2.066942e+12
2024,734,2.749008e+09,2.994371e+12
2025,1000,3.799224e+09,3.785304e+12


### Build union of tickers and show sample

In [32]:
all_tickers = set()
for df in year2blg.values():
    all_tickers.update(df.index.tolist())

tickers = sorted(all_tickers)
print(f"Total unique tickers across all years: {len(tickers)}")

tickers[:20]

Total unique tickers across all years: 1303


['A',
 'AA',
 'AAL',
 'AAP',
 'AAPL',
 'ABBV',
 'ABCB',
 'ABG',
 'ABNB',
 'ABT',
 'ACA',
 'ACC',
 'ACGL',
 'ACHC',
 'ACI',
 'ACIW',
 'ACM',
 'ACN',
 'AD',
 'ADBE']

### Build daily excess returns panel

In [ ]:
%%time

returns_panel = data_pipeline.build_returns_panel(
    tickers=tickers,
    use_excess= True,
)

returns_panel.info()

In [44]:
returns_panel.head()

,A,AA,AAL,AAP,AAPL,...,ZION,ZM,ZNGA,ZS,ZTS
Date,,,,,,,,,,,
2016-01-04,-0.026798,-0.016221,-0.034013,0.011484,0.000845,...,-0.021622,NaN,NaN,NaN,-0.013575
2016-01-05,-0.003451,-0.045324,-0.009544,-0.006841,-0.025070,...,-0.010867,NaN,NaN,NaN,0.015644
2016-01-06,0.004429,-0.071208,0.017512,-0.026465,-0.019580,...,-0.029155,NaN,NaN,NaN,0.000198
2016-01-07,-0.042485,-0.039499,-0.018929,0.011063,-0.042215,...,-0.030030,NaN,NaN,NaN,-0.030414
2016-01-08,-0.010523,-0.024194,-0.001988,-0.021982,0.005278,...,-0.011265,NaN,NaN,NaN,-0.014615


In [46]:
returns_panel.to_parquet(panel_path,engine="fastparquet")
panel_path

PosixPath('data/processed/daily_excess_returns.parquet')

In [45]:
returns_panel.describe().T.head(20)

,count,mean,std,min,25%,50%,75%,max
A,2499.0,0.000588,0.017336,-0.110209,-0.007947,0.001002,0.009724,0.098382
AA,2499.0,0.000788,0.034567,-0.210729,-0.018161,-0.000061,0.019104,0.282175
AAL,2499.0,0.000056,0.033069,-0.252258,-0.015265,-0.000263,0.014447,0.410964
AAP,2499.0,-0.000110,0.027609,-0.350556,-0.011230,-0.000213,0.011535,0.570260
AAPL,2499.0,0.001071,0.018346,-0.128658,-0.007208,0.000982,0.009972,0.153125
...,...,...,...,...,...,...,...,...
ACIW,2499.0,0.000470,0.022250,-0.186189,-0.010539,0.000508,0.011321,0.157251
ACM,2499.0,0.000611,0.019491,-0.168874,-0.009024,0.000798,0.009741,0.146786
ACN,2499.0,0.000495,0.016188,-0.093279,-0.006748,0.001127,0.008413,0.128562
AD,2499.0,0.000587,0.032033,-0.224316,-0.010901,0.000731,0.011516,0.928938


In [ ]:
### EOF